In [2]:
from build123d import *
from jupyter_cadquery import open_viewer, set_defaults, show, get_defaults
set_defaults(theme="dark")
cv = open_viewer("CadQuery", anchor="right", glass=True)
set_defaults(reset_camera=False)

Overwriting auto display for cadquery Workplane and Shape


In [36]:
AXIS_D = 2
TH = 1
H = 7


rodholder = Cylinder(AXIS_D/2+TH, H) - Pos(0,0, TH) * Cylinder(AXIS_D/2+0.2, H) \
+ Pos(5/2, (TH-0.2)/2+AXIS_D/2+0.2) * Box(5, TH-0.2, H) \
+ Pos(6/2 + 5-2, TH/2+AXIS_D/2+TH) * Box(6, TH, H)

rodholder.export_stl('/tmp/rodholder.stl')
rodholder

In [146]:
# Main SURFACE
TA = (Align.CENTER, Align.CENTER, Align.MIN)
MAIN_SURFACE_Z = 1.5
MAIN_SURFACE_X = 99
MAIN_SURFACE_Y = 85

HOUSING_RADIUS = 10

# mounting studs for drive unit
STUD0 = [10.5 - 100/2, 40 - 85/2]
STUD1 = [84.5 - 100/2, 7.5 - 85/2]
STUD2 = [81.5 - 100/2, 79 - 85/2]

STUD_RUBBER_D = 7
STUD_H = 3
STUD_D = 3.5
STUD_OFFSET_Z = 0.6 # The stud starts 1mm above the main surface
STUD_DRILL_D = 1.5


OFFSET_DRIVE = Pos(2.5,0)

POS_DRIVE = OFFSET_DRIVE * Pos(47.5, 38) * Rot(180,0,90) # we need to rotate and shift the data from the drawing of the drive
DRIVE_OUTLINE = POS_DRIVE * extrude(make_face(import_svg('drawing_drive_mechanics_without_holes.svg')), 10, both=True)
STUDS = [OFFSET_DRIVE * Pos(p) for p in [STUD0, STUD1, STUD2]]

#+ Pos(MAIN_SURFACE_X/2-1/2-1, 0) * Box(1, MAIN_SURFACE_Y, MAIN_SURFACE_Z, align=TA) \
drive_base = fillet(Box(MAIN_SURFACE_X, MAIN_SURFACE_Y, MAIN_SURFACE_Z, align=TA).edges() | Axis.Z, HOUSING_RADIUS) \
- DRIVE_OUTLINE \
- Pos(0,0,STUD_OFFSET_Z) * STUDS * Cylinder((STUD_RUBBER_D+0.5)/2, 10, align=TA) \
+ Pos(0,0,STUD_OFFSET_Z) * STUDS * Cylinder(STUD_D/2, STUD_H, align=TA) \
- Pos(0,0,STUD_OFFSET_Z) * STUDS * Cylinder(STUD_DRILL_D/2, STUD_H, align=TA)

drive_base.export_stl('/tmp/drive_base.stl')
drive_base

In [147]:
# housing shell
SHELL_Z = 10 # measured from Z=0
SHELL_TH = 1

shell = fillet(Box(MAIN_SURFACE_X, MAIN_SURFACE_Y, SHELL_Z, align=TA).edges() | Axis.Z, HOUSING_RADIUS)
shell = offset(shell, -SHELL_TH, shell.faces() | Axis.Z)

shell

In [161]:
# mount of drive electronics
STUDS_DRIVE_POS = [[58-101/2+6, 82-85/2-1-2], [6-101/2+6, 14-85/2-2.5], [18-101/2+6, 63-85/2-2]]
STUDS_DRIVE_H = 3 #above main surface
STUDS_DRIVE_DRILL_D = 1.5

studs_drive = [Pos(p) for p in STUDS_DRIVE_POS] * (Cylinder(5/2, STUDS_DRIVE_H+MAIN_SURFACE_Z, align=TA) - Cylinder(STUDS_DRIVE_DRILL_D/2, STUDS_DRIVE_H+MAIN_SURFACE_Z, align=TA))


HINGE_X = 5
HINGE_Y = 5
HINGE_HOLDER_X = 5
HINGE_HOLDER_Y = 5
HINGE_SPHERE_D = 2.5
HINGE_HEIGHT = SHELL_Z - HINGE_SPHERE_D/2 - 1
HINGE0_X = -MAIN_SURFACE_X/2 + HOUSING_RADIUS + HINGE_HOLDER_X
HINGE1_X = MAIN_SURFACE_X/2 - HOUSING_RADIUS - HINGE_HOLDER_X - 12
hinge_opening = Box(HINGE_X+0.6, HINGE_Y+0.6, SHELL_Z, align=(Align.MIN, Align.MAX, Align.MIN))
hinge_holder = Box(HINGE_HOLDER_X, HINGE_HOLDER_Y, SHELL_Z, align=(Align.MAX, Align.MAX, Align.MIN)) + Pos(0, -HINGE_HOLDER_Y/2, HINGE_HEIGHT) * Sphere((HINGE_SPHERE_D)/2)

housing = drive_base + shell + studs_drive
#add hinges
housing = housing - Pos(HINGE0_X, MAIN_SURFACE_Y/2) * hinge_opening + Pos(HINGE0_X, MAIN_SURFACE_Y/2) * hinge_holder
housing = housing - Pos(HINGE1_X, MAIN_SURFACE_Y/2) * mirror(hinge_opening, Plane.YZ) + Pos(HINGE1_X, MAIN_SURFACE_Y/2) * mirror(hinge_holder, Plane.YZ)


# mount of main PCB
PCB_OFFSET_X = 1.5
PCB_OFFSET_Y = -6
STUDS_PCB_POS = [[4, -4.75], [2.25, -22], [7.25, -73.25], [32.25, -73.25], [77.25, -75.5], [93.25, -71.25], [71.25, -1.25], [90.25, -2.25], [2.25, -42.25]]
STUDS_PCB_H = 3 #above main surface
STUDS_PCB_D = 3.5
STUDS_PCB_DRILL_D = 1.5
stud_pcb = Cylinder(STUDS_PCB_D/2, STUDS_PCB_H+MAIN_SURFACE_Z, align=TA) - Cylinder(STUDS_PCB_DRILL_D/2, STUDS_PCB_H+MAIN_SURFACE_Z, align=TA)
studs_pcb = [Pos(-MAIN_SURFACE_X/2+PCB_OFFSET_X, MAIN_SURFACE_Y/2+PCB_OFFSET_Y) * Pos(p) for p in STUDS_PCB_POS] * stud_pcb

housing += studs_pcb


housing.export_stl('/tmp/housing.stl')
housing

In [164]:
#lid
LID_INNER_H = 3.5
LID_TH = 1

lid = fillet(Box(MAIN_SURFACE_X, MAIN_SURFACE_Y, LID_INNER_H+LID_TH, align=TA).edges() | Axis.Z, HOUSING_RADIUS)
lid = chamfer(lid.edges().group_by()[0], 3)

DISK_CENTER_X = -MAIN_SURFACE_X/2 + 41.8
DISK_CENTER_Y = -MAIN_SURFACE_Y/2 + 42.8
SPINDLE_D = 17

lid = lid + Pos(DISK_CENTER_X, DISK_CENTER_Y) * Cylinder(83/2, LID_INNER_H+LID_TH, align=TA)

lid = offset(lid, -LID_TH, lid.faces().group_by()[-1])

top_hinge_plane =  Plane.XY.offset(LID_INNER_H+LID_TH+SHELL_Z)

hinges_drawing =  top_hinge_plane * (Pos(HINGE0_X+0.2, MAIN_SURFACE_Y/2) * Rectangle(HINGE_X, HINGE_Y, align=(Align.MIN, Align.MAX)) + Pos(HINGE1_X-0.2, MAIN_SURFACE_Y/2) * Rectangle(HINGE_X, HINGE_Y, align=(Align.MAX, Align.MAX)))
hinges = extrude(hinges_drawing, until=Until.NEXT, target=lid, dir=(0,0,-1)) - Pos(0,MAIN_SURFACE_Y/2-HINGE_Y/2,HINGE_HEIGHT+LID_TH+LID_INNER_H) * Rot(0,90,0) * Cylinder((HINGE_SPHERE_D)/2, MAIN_SURFACE_X)
lid = lid + hinges - Pos(DISK_CENTER_X,DISK_CENTER_Y,0.5) * Cylinder(SPINDLE_D/2, LID_TH, align=TA)

#round off one side of the hinges
lid = fillet((lid.edges().group_by()[-1] | Axis.X).group_by(Axis.Y)[-1], 3)


lid.export_stl('/tmp/lid.stl')
lid

In [154]:
pcb = import_step('/tmp/minidisc.step')
pcb = Pos(-MAIN_SURFACE_X/2+PCB_OFFSET_X, MAIN_SURFACE_Y/2+PCB_OFFSET_Y, MAIN_SURFACE_Z+STUDS_PCB_H)*pcb
pcb.export_stl('/tmp/pcb.stl')
pcb

In [155]:
assembly = Compound(label='assembly', children=[housing, Pos(0,0,-(LID_INNER_H+LID_TH)) * lid, pcb])
assembly.color = Color('red')
assembly


100% ⋮————————————————————————————————————————————————————————————⋮ (3/3)  0.36s
